In [1]:
import tensorflow as tf
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import scipy.sparse as sp
import pandas as pd
import pickle
from datetime import timedelta
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
#from tensorflow.contrib.rnn.python.ops import rnn_cell as RNNCell
from tensorflow.python.ops.rnn_cell_impl import _RNNCell as RNNCell
import collections
from tensorflow.contrib import rnn
import h5py
#from tensorflow.python.ops.rnn_cell_impl import _RNNCell 
import random
import pickle

In [2]:
def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    #adj[np.isnan(adj)] = 0.
    adj = tf.abs(adj)
    rowsum = tf.reduce_sum(adj, 1)# sum by row

    d_inv_sqrt = tf.pow(rowsum, -0.5)
   
    #d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    
    d_mat_inv_sqrt = tf.diag(d_inv_sqrt)

    return tf.matmul(tf.matmul(d_mat_inv_sqrt, adj), d_mat_inv_sqrt)

def masked_mae_tf(preds, labels, null_val=np.nan):
    """
    Accuracy with masking.
    :param preds:
    :param labels:
    :param null_val:
    :return:
    """
    #print (preds.shape)
    #print (labels.shape)
    if np.isnan(null_val):
        mask = ~tf.is_nan(labels)
    else:
        mask = tf.not_equal(labels, null_val)
    mask = tf.cast(mask, tf.float32)
    mask /= tf.reduce_mean(mask)
    mask = tf.where(tf.is_nan(mask), tf.zeros_like(mask), mask)
    loss = tf.abs(tf.subtract(preds, labels))
    loss = loss * mask
    loss = tf.where(tf.is_nan(loss), tf.zeros_like(loss), loss)
    return tf.reduce_mean(loss)

def masked_mae_tf_by_horizon(preds, labels, null_val=np.nan):
    """
    Accuracy with masking.
    :param preds:
    :param labels:
    :param null_val:
    :return:
    """
    preds_reshape = tf.reshape(preds, [-1, sn, horizon])
    labels_reshape = tf.reshape(labels, [-1, sn, horizon])
    
    res = []
    
    for i in range(horizon):
        labels = labels_reshape[:, :, 0:(i+1)]
        preds = preds_reshape[:, :, 0:(i+1)]
        
        if np.isnan(null_val):
            mask = ~tf.is_nan(labels)
        else:
            mask = tf.not_equal(labels, null_val)
        mask = tf.cast(mask, tf.float32)
        mask /= tf.reduce_mean(mask)
        mask = tf.where(tf.is_nan(mask), tf.zeros_like(mask), mask)
        loss = tf.abs(tf.subtract(preds, labels))
        loss = loss * mask
        loss = tf.where(tf.is_nan(loss), tf.zeros_like(loss), loss)
        
        res.append(tf.reduce_mean(loss))
        
    return res

In [3]:
class StandardScaler:
    """
    Standard the input
    """

    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def transform(self, data):
        return (data - self.mean) / self.std

    def inverse_transform(self, data):
        return (data * self.std) + self.mean


In [4]:
file_Name = "../../data/nyc_bike/NYCBikeHourly272.pickle"
fileObject = open(file_Name,'rb') 
hourly_bike = pickle.load(fileObject) 
hourly_bike = pd.DataFrame(hourly_bike)
raw_data = pd.DataFrame(hourly_bike)

In [5]:
raw_data.shape

(26304, 272)

In [6]:
raw_data.head()

,0,1,2,3,4,5,6,7,8,9,...,262,263,264,265,266,267,268,269,270,271
0,2,1,0,3,0,0,0,4,1,2,...,3,1,1,0,0,0,0,0,1,0
1,1,0,0,1,2,1,0,0,0,0,...,0,2,1,0,5,0,0,0,0,0
2,0,0,0,0,0,3,0,0,0,0,...,1,1,0,0,2,0,0,0,0,1
3,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# revised based on https://github.com/transpaper/gconvRNN/blob/master/model.py

def conv(x, ddgf, feat_out, K, W):
    '''
    x : [batch_size, N_node, feat_in] - input of each time step
    nSample : number of samples = batch_size, let's say it is 100
    nNode : number of node in graph
    feat_in : number of input feature, usually is set as 1
    feat_out : number of output feature
    ddgf : data driven graph filter
    K : size of kernel(number of cheby coefficients), is 1 in the fast graph paper
    W : cheby_conv weight [K * feat_in, feat_out]
    '''
    if len(x.shape) == 2:
        x = tf.expand_dims(x, 2) # extend a dimension "feature_in"
    nSample, nNode, feat_in = x.get_shape()
    #feat_in = 1
    print (nSample, nNode, feat_in)
    nSample, nNode, feat_in = int(nSample), int(nNode), int(feat_in)
    
    x0 = tf.transpose(x, perm=[1, 2, 0]) #change it to [nNode, feat_in, nSample]
    x0 = tf.reshape(x0, [nNode, feat_in*nSample]) # feature_in = 1, [nNode, nSample]
    #x = tf.expand_dims(x0, 0) # make it [1, nNode, feat_in*nSample]
    
    x0 = tf.matmul(ddgf, x0) # graph convolutional #[nNode, nSample]
        
    #x = tf.reshape(x, [K, nNode, feat_in, nSample])
    #x = tf.transpose(x, perm=[3,1,2,0])
    x0 = tf.reshape(x0, [nSample*nNode, feat_in*K]) #[nSample*nNode, 1]
    
    x = tf.matmul(x0, W) #No Bias term?? -> Yes
    out = tf.reshape(x, [nSample, nNode, feat_out]) 
    return out


In [8]:
# Create model
def gcn(x, weights, biases, batch_size, n_input, frequency,flag, n_output_vec):
    # Hidden layer with RELU activation
    
   # output_list = tf.Variable(tf.zeros([n_output_vec,1]),dtype=tf.float32) #'Tensor' object does not support item assignment, cant build Ypre

    #Xtem = tf.reshape(x[i,:], [n_input, frequency])
    #Xtem = tf.transpose(Xtem)
    #Atem = tf.convert_to_tensor(A_whole_final, dtype=np.float32)
    #Atem1 = tf.diag(tf.ones([n_input])) # Atem not palying any roles
    #Ytem = tf.reshape(Y[i,:], [n_input, 1])
    #Atem = tf.diag(tf.ones([n_input]))
    #x = tf.reshape(x, [-1, sn, 1]) # 100, 207, 1
    
    # x (?, 207, 12)
    x = tf.transpose(x, [1, 0, 2]) # 207, ?, 12
    x = tf.reshape(x, [sn, -1]) # 207, batch*feature_num
    Atem1 = 0.5*(weights['A1'] + tf.transpose(weights['A1']))#+ Atem 
    Atem1 = normalize_adj(Atem1)
    #th = tf.constant(0.01, dtype=tf.float32)
    #where = tf.subtract(Atem1, th)
    #Atem1 = tf.nn.relu(where)

    Z1 = tf.matmul(Atem1, x) # 207, batch*feature_num  #+ tf.matmul( tf.matmul(weights['A1'], weights['A1']), Xtem)
    Z1 = tf.reshape(Z1, [-1, frequency]) # 207* 100, frequency
    #layer_1 = tf.matmul(Z1, weights['h1']) 
    layer_1 = tf.add(tf.matmul(Z1, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1) # 207*100, hidden1

    
    #Atem2 = 0.5*(weights['A2'] + tf.transpose(weights['A2']))#+ Atem 
    #Atem2 = normalize_adj(Atem2)
    
    #layer_1 = tf.reshape(layer_1, [sn, -1])  # 207, batchsize*hidden1
    #Z2 = tf.matmul(Atem2, layer_1)
    #Z2 = tf.reshape(Z2, [-1, n_hidden_vec1]) # 207*batchsize, n_hidden_vec1
    #layer_2 = tf.add(tf.matmul(Z2, weights['h2']), biases['b2'])
    #layer_2 = tf.nn.relu(layer_2) # 207*batchsize, hidden2

    #Atem3 = 0.5*(weights['A3'] + tf.transpose(weights['A3']))#+ Atem 
    #Atem3 = normalize_adj(Atem3) 
    
    #layer_2 = tf.reshape(layer_2, [sn, -1])  # 207, batchsize*hidden2
    #Z3 = tf.matmul(Atem3, layer_2)
    #Z3 = tf.reshape(Z3, [-1, n_hidden_vec2]) # 207*batchsize, hidden2
    #layer_3 = tf.add(tf.matmul(Z3, weights['h3']), biases['b3'])
    #layer_3 = tf.nn.relu(layer_3) # 207*batchsize, hidden3
    
    #Atem4 = 0.5*(weights['A4'] + tf.transpose(weights['A4']))#+ Atem 
    #Atem4 = normalize_adj(Atem4)
    #Z4 = tf.matmul(Atem4, layer_3)
    #layer_4 = tf.add(tf.matmul(Z4, weights['h4']), biases['b4'])
    #layer_4 = tf.nn.relu(layer_4)

    # flattern
    #layer_3 = tf.reshape(layer_3, [1, 272*n_hidden_vec3])

    #F1 = tf.add(tf.matmul(layer_3, weights['f1']), biases['bf1'])
    #F1 = tf.nn.relu(F1)

    #F2 = tf.add(tf.matmul(F1, weights['f2']), biases['bf2'])
    #F2 = tf.nn.relu(F2)

    #F3 = tf.add(tf.matmul(F1, weights['f3']), biases['bf3'])
    #out_layer = tf.reshape(F3, [272, 1])

    # Output layer with linear activation
    layer_1 = tf.reshape(layer_1, [sn, -1, n_hidden_vec1])
    layer_1 = tf.transpose(layer_1, [1, 0, 2]) # batchsize, sn, hidden3
    layer_1 = tf.reshape(layer_1, [-1, sn*n_hidden_vec1]) # batchsize, sn*hidden3
    Z4 = layer_1
    #out_layer = tf.add(tf.matmul(Z4, weights['outg']), biases['boutg'])
    #tf.matmul(Atem1, layer_1)#tf.matmul(Atem, layer_3)
    #out_layer = tf.add(tf.matmul(Z4, weights['out']), biases['bout'])
    #out_layer = tf.nn.relu(out_layer)
    # weather layer 1
    #x_wea_tem = tf.reshape(x_wea[i,:], [1, 9*frequency2]) # 1 by 126
    #layer_1_wea = tf.add(tf.matmul(x_wea_tem, weights['h1_wea']), biases['b1_wea'])
    #layer_1_wea = tf.nn.relu(layer_1_wea)

    #out_layer_wea = tf.add(tf.matmul(layer_1_wea, weights['out_wea']), biases['bout_wea'])
    #out_layer = tf.add(out_layer, tf.reshape(out_layer_wea, [272, 1]))


    
    return Z4

In [9]:
def gcn_corr_final(frequency, horizon, learning_rate, decay,batch_size, n_hidden_vec1,n_hidden_vec2,n_hidden_vec3,keep, early_stop_th,training_epochs, reg1, reg2):
    # set size
    #sn = 3 # station number
    X_whole = []
    Y_whole = []

    x_offsets = np.sort(
        # np.concatenate(([-week_size + 1, -day_size + 1], np.arange(-11, 1, 1)))
        np.concatenate((np.arange(-frequency+1, 1, 1),))
    )
    # Predict the next one hour
    y_offsets = np.sort(np.arange(1, 1+ horizon, 1))

    min_t = abs(min(x_offsets))
    max_t = abs(raw_data.shape[0] - abs(max(y_offsets)))  # Exclusive
    for t in range(min_t, max_t):
        x_t = raw_data.iloc[t + x_offsets, 0:sn].values.flatten('F')
        y_t = raw_data.iloc[t + y_offsets, 0:sn].values.flatten('F')
        X_whole.append(x_t)
        Y_whole.append(y_t)

    X_whole = np.stack(X_whole, axis=0)
    time_step = int(X_whole.shape[1] / sn)
    X_whole = np.reshape(X_whole, [X_whole.shape[0], sn, time_step])
    Y_whole = np.stack(Y_whole, axis=0)
    
    i = lstm_steps
    X_whole_lstm = []
    Y_whole_lstm = []
    
    while i < X_whole.shape[0]:
        X_whole_lstm.append(X_whole[i-lstm_steps:i,:])
        Y_whole_lstm.append(Y_whole[i])
        i = i + 1
    
    X_whole_lstm = np.stack(X_whole_lstm, axis = 0) # (34239, 10, 207, 12)
    Y_whole_lstm = np.stack(Y_whole_lstm, axis = 0) # (34239, 2484)
    #print (Y_whole_lstm.shape)
    '''
    time_step = int(time_step) #

    #i = time_step
    #X_whole = np.zeros(shape = (raw_data.shape[0] - time_step, sn*time_step), dtype = np.float)
    #Y_whole = np.zeros(shape = (raw_data.shape[0] - time_step, sn), dtype = np.float)

    while i < raw_data.shape[0]:
        X_whole[i - time_step, ] = raw_data.iloc[(i - time_step):i, 0:sn].values.flatten('F') # 'F' flatten by column, default:flatten by row 0, 1, 2...7
        Y_whole[i - time_step, ] = raw_data.iloc[i, 0:sn]
        i = i + 1
    '''


    n_input = sn # station number
    n_input_vec = n_input * frequency # 207 * frequency
    n_A_vec = n_input * n_input
    n_output_vec = Y_whole_lstm.shape[1] # each row represent a result
    #print (n_output_vec)

    
    num_samples = X_whole_lstm.shape[0]
    num_train = 20000 # Note here actually we use the first 20000 to train the model. The paper mentioned "22304" need to be corrected.
    num_val = 2000
    num_test = 2000
    #skip = skip1 + freq_max - time_step#time_step_max - time_step # to make sure the testing datasets are the same although the frequency could be different

    X_training = X_whole_lstm[:num_train, :]
    Y_training = Y_whole_lstm[:num_train, :]
    
    # shuffle
    perm = np.arange(X_whole_lstm.shape[0])
    np.random.shuffle(perm)
    X_training = X_whole_lstm[perm]
    Y_training = Y_whole_lstm[perm]
    
    #print (type(X_training))
    #X_training = random.Random(6).shuffle(X_training)
    #Y_training = random.Random(6).shuffle(Y_training)

    X_val = X_whole_lstm[num_train:num_train+num_val, :]
    Y_val = Y_whole_lstm[num_train:num_train+num_val, :]
    #A_val = A_whole[0+training:0+training+validation, :]

    X_test = X_whole_lstm[num_train+num_val:num_train+num_val+num_test, :]
    Y_test = Y_whole_lstm[num_train+num_val:num_train+num_val+num_test, :]

    scaler = StandardScaler(mean=X_training.mean(), std=X_training.std())

    X_training = scaler.transform(X_training)
    Y_training = scaler.transform(Y_training)

    X_val = scaler.transform(X_val)
    Y_val = scaler.transform(Y_val)

    X_test = scaler.transform(X_test)
    Y_test = scaler.transform(Y_test)
    
    early_stop_th = int(early_stop_th)
    training_epochs = int(training_epochs)
    
    early_stop_k=0
    display_step = 1
    best_val = 10000
    traing_error = 0
    test_error = 0
    predic_res = []
    Y_true = []

    tf.reset_default_graph()

    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)
    
    
    batch_size = int(batch_size)
    n_hidden_vec1 = int(n_hidden_vec1)
    n_hidden_vec2 = int(n_hidden_vec2)
    n_hidden_vec3 = int(n_hidden_vec3)
    early_stop_th = int(early_stop_th)
    training_epochs = int(training_epochs)
    
    early_stop_k=0
    display_step = 1
    best_val = 10000
    traing_error = 0
    test_error = 0
    # Network Parameters

    #n_classes = 2 # MNIST total classes (0-9 digits) # n_classes is for classification only

    tf.reset_default_graph()

    # tf Graph input
    X = tf.placeholder(tf.float32, [None, lstm_steps, sn, time_step]) # X is the input signal
    #X_weather = tf.placeholder(tf.float32, [None, 9 * frequency2]) # X_weather weather and holiday information (9 is the feature number)
    A = tf.placeholder(tf.float32, [None, n_A_vec]) # A is the normalized adj matrix
    oldA = tf.placeholder(tf.float32, [n_input, n_input])
    Y = tf.placeholder(tf.float32, [None, n_output_vec]) # y is the regression output
    #num = tf.placeholder(tf.int32,[1, 1] )

    #Xtem = tf.placeholder(tf.float32, [n_input, frequency]) # for each row of X, A, Y, it can be reshaped to Xtem, Atem, Ytem
    #Atem = tf.placeholder(tf.float32, [n_input, n_input]) # 
    #Ytem = tf.placeholder(tf.float32, [n_input, 1]) #

    #Ypre = tf.placeholder(tf.float32, [None, n_output_vec])

    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([frequency, n_hidden_vec1]), dtype=np.float32),
        'h2': tf.Variable(tf.random_normal([n_hidden_vec1, n_hidden_vec2]), dtype=np.float32),
        'h3': tf.Variable(tf.random_normal([n_hidden_vec2, n_hidden_vec3]), dtype=np.float32),
        #'h4': tf.Variable(tf.random_normal([n_hidden_vec3, n_hidden_vec4])),
        'outg': tf.Variable(tf.random_normal([sn*n_hidden_vec1, n_hidden_vec4]), dtype=np.float32), 
        'out': tf.Variable(tf.random_normal([num_hidden, Y_whole.shape[1]]), dtype=np.float32), # dont forget to change n_hidden_vec1 when add/delete layers
        #'f1': tf.Variable(tf.random_normal([272*n_hidden_vec3, 100])),
        #'f2': tf.Variable(tf.random_normal([50, 10])),
        #'f3': tf.Variable(tf.random_normal([100, 272])),
        'A1': tf.Variable(tf.random_normal([n_input,n_input]), dtype=np.float32),
        'A2': tf.Variable(tf.random_normal([n_input,n_input]), dtype=np.float32),
        'A3': tf.Variable(tf.random_normal([n_input,n_input]), dtype=np.float32),
        #'A4': tf.Variable(tf.random_normal([n_input,n_input])),
        #'h1_wea': tf.Variable(tf.random_normal([9*frequency2, n_hidden_weather1])),
        #'out_wea': tf.Variable(tf.random_normal([n_hidden_weather1, n_input]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([1, n_hidden_vec1]), dtype=np.float32),# n_hidden_vec1])),# bias all the same??? dont forget to test
        'b2': tf.Variable(tf.random_normal([1, n_hidden_vec2]), dtype=np.float32), #n_hidden_vec2])),
        'b3': tf.Variable(tf.random_normal([1, n_hidden_vec3]), dtype=np.float32),#n_hidden_vec3])),
        #'b4': tf.Variable(tf.random_normal([n_input, n_hidden_vec4])),
        #'b1': tf.Variable(tf.random_normal([n_input,n_hidden_vec1])),# bias all the same??? dont forget to test
        #'b2': tf.Variable(tf.random_normal([n_input,n_hidden_vec2])),
        #'b3': tf.Variable(tf.random_normal([n_input,n_hidden_vec3])),
        #'bf1': tf.Variable(tf.random_normal([1, 100])), 
        #'bf2': tf.Variable(tf.random_normal([1, 10])), 
        #'bf3': tf.Variable(tf.random_normal([1, 272])), 
        'boutg': tf.Variable(tf.random_normal([1, n_hidden_vec4]), dtype=np.float32), 
        'bout': tf.Variable(tf.random_normal([Y_whole.shape[1]]), dtype=np.float32), 
        #'b1_wea': tf.Variable(tf.random_normal([1, n_hidden_weather1])), 
        #'bout_wea': tf.Variable(tf.random_normal([1, n_input])), 
    }
    
    with tf.variable_scope('lstm'):
        lstm = tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(num_hidden, forget_bias=1.0, state_is_tuple=True)
        rnn_input_seq = tf.unstack(X, lstm_steps, 1) # lstm_steps is the 2nd variable

        for i in range(lstm_steps):
            rnn_input_seq[i] = gcn(rnn_input_seq[i], weights, biases, batch_size,n_input, frequency, 1, n_output_vec)
            #print (rnn_input_seq[i].shape)
        outputs, states = tf.contrib.rnn.static_rnn(lstm, rnn_input_seq, dtype=tf.float32)
        output_reshape = tf.reshape(outputs[-1], [-1, num_hidden])
        #print ('123here!!!!!!!!!!!')
        pred = tf.matmul(output_reshape, weights['out']) + biases['bout']
        #print (pred)
        #pred = tf.reshape(pred, [-1, Y_whole.shape[1]])
        #print ('here!!!!!!!!!!!')
        pred = scaler.inverse_transform(pred)
        Y_true_tr = scaler.inverse_transform(Y)

        cost = tf.reduce_mean(tf.pow(pred - Y_true_tr, 2)) 
        #print (cost)
        
    with tf.variable_scope('lstm', reuse=True):
        rnn_input_seq_val = tf.unstack(X, lstm_steps, 1)

        for i in range(lstm_steps):
            rnn_input_seq_val[i] = gcn(rnn_input_seq_val[i], weights, biases, batch_size,n_input,frequency, 2, n_output_vec)
        outputs_val, states = tf.contrib.rnn.static_rnn(lstm, rnn_input_seq_val, dtype=tf.float32)
        output_reshape = tf.reshape(outputs_val[-1], [-1, num_hidden])

        pred_val = tf.matmul(output_reshape, weights['out']) + biases['bout']
        pred_val = scaler.inverse_transform(pred_val)
        Y_true_val = scaler.inverse_transform(Y)
        
        cost_val =  tf.reduce_mean(tf.pow(pred_val - Y_true_val, 2)) 
        #print ('234here!!!!!!!!!!!')
    with tf.variable_scope('lstm', reuse=True):
        rnn_input_seq_test = tf.unstack(X, lstm_steps, 1)
        
        for i in range(lstm_steps):
            rnn_input_seq_test[i] = gcn(rnn_input_seq_test[i], weights, biases, batch_size,n_input,frequency, 3, n_output_vec)
        outputs_test, states = tf.contrib.rnn.static_rnn(lstm, rnn_input_seq_test, dtype=tf.float32)
        output_reshape = tf.reshape(outputs_test[-1], [-1, num_hidden])

        pred_tes = tf.matmul(output_reshape, weights['out']) + biases['bout']
        pred_tes = scaler.inverse_transform(pred_tes)
        Y_true_tes = scaler.inverse_transform(Y)
        
        cost_tes = tf.reduce_mean(tf.pow(pred_tes - Y_true_tes, 2)) 
        
        #print ('345here!!!!!!!!!!!')
    #rmse
    #cost_tes = tf.reduce_mean(tf.pow(pred_tes-Y, 2))
    # cross-entropy for classification
    # cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y_train))
    # ratio = tf.abs(tf.reduce_sum(pred)-tf.reduce_sum(Y))/tf.reduce_sum(Y)
    #zero = 0
    #ratio = tf.reduce_mean(tf.divide(tf.where(tf.not_equal(Y, zero), np.abs(pred-Y), tf.zeros(Y.get_shape(), tf.float32)), tf.where(tf.not_equal(Y, zero), Y, tf.ones(Y.get_shape(), tf.float32))))
    optimizer = tf.train.RMSPropOptimizer(learning_rate, decay).minimize(cost)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Initializing the variables
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    #total_val_cost = []
    #total_val_ratio = []

    # learning start from 

    #index = daily_bike[(daily_bike['year'] == 2016) & (daily_bike['monthofyear'] == 1) & (daily_bike['dayofmonth'] == 1)].index.tolist()[0]
    #A_hat = normalize_adj(corr_matrix_trips)
    #print(A_hat)
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    with tf.Session(config=config) as sess:
        sess.run(init)

        for epoch in range(training_epochs):

            avg_cost = 0.
            total_batch = int(num_train/batch_size) #int(num_train/batch_size)

            for i in range(total_batch):
                #print (Y_training[i*batch_size:(i+1)*batch_size,].size())
                #num = batch_size
                _, c, preds, trueval = sess.run([optimizer, cost, pred, Y_true_tr], feed_dict={X: X_training[i*batch_size:(i+1)*batch_size,], 
                                                      Y: Y_training[i*batch_size:(i+1)*batch_size,],  
                                                              keep_prob: keep})
                #print (preds)
                #print (trueval)
                #print ("Epoch:", '%04d' % (epoch+1), "batch: ", i, "batch cost=", \
                #    "{:.9f}".format(c))
                #print ('here!!!!!!!!!!!!!!!!')
                avg_cost += c * batch_size #/ total_batch 
                #Display logs per epoch step
                
            # rest part of training dataset
            #num = num_train - total_batch*batch_size 
            if total_batch * batch_size != num_train:
                _, c, preds, trueval = sess.run([optimizer, cost, pred, Y_true_tr], feed_dict={X: X_training[total_batch*batch_size:num_train,], 
                                          Y: Y_training[total_batch*batch_size:num_train,],
                                                  keep_prob: keep})
                avg_cost += c * (num_train - total_batch*batch_size)
            
            avg_cost = np.sqrt(avg_cost / num_train)
            
            if epoch % display_step == 0:
                print ("Epoch:", '%04d' % (epoch+1), "Training RMSE=", \
                    "{:.9f}".format(avg_cost)) #np.sqrt(avg_cost)
                
            # also use batch to save memory
            # validation
            c_val = 0.
            total_bat_val = int(num_val/batch_size)
            for i in range(total_bat_val):
                #num = batch_size
                c_val_b = sess.run([cost_val], feed_dict={X: X_val[i*batch_size:(i+1)*batch_size,], 
                                                          Y: Y_val[i*batch_size:(i+1)*batch_size,],   keep_prob:1})
                c_val += c_val_b[0]*batch_size
            
            if total_bat_val * batch_size != num_val:
                #num = num_val - total_bat_val*batch_size
                c_val_b = sess.run([cost_val], feed_dict={X: X_val[total_bat_val*batch_size:num_val,], 
                                                          Y: Y_val[total_bat_val*batch_size:num_val,],  keep_prob:1})
                c_val += c_val_b[0] * (num_val - total_bat_val*batch_size)
                
            c_val = np.sqrt(c_val / num_val)
            
            print("Validation RMSE: ", c_val)
            
            # test
            c_tes = 0.
            total_bat_test = int(num_test/batch_size)
            
            pre_test_tem = [] # save the prediction results
            Y_tes_true = []
            
            for i in range(total_bat_test):
                #num = batch_size
                c_tes_b, pred_tes1, Y_tes_batch = sess.run([cost_tes, pred_tes, Y_true_tes], feed_dict={X: X_test[i*batch_size:(i+1)*batch_size,],
                                                                               Y: Y_test[i*batch_size:(i+1)*batch_size,],  keep_prob: 1})
                c_tes += c_tes_b*batch_size

                #print (cost_h)
                #print (pred_tes1.shape)
                pre_test_tem.append(pred_tes1)
                Y_tes_true.append(Y_tes_batch)
                
            if total_bat_test * batch_size != num_test:
                #num = num_test - total_bat_test*batch_size
                c_tes_b, pred_tes1, Y_tes_batch = sess.run([cost_tes, pred_tes, Y_true_tes], feed_dict={X: X_test[total_bat_test*batch_size:num_test,],
                                                                               Y: Y_test[total_bat_test*batch_size:num_test,],  keep_prob: 1})
                c_tes += c_tes_b * (num_test - total_bat_test*batch_size) 
                 
                
                #print (pred_tes1.shape)
                pre_test_tem.append(pred_tes1)
                Y_tes_true.append(Y_tes_batch)
            
            #print (c_tes_h.shape)
            pre_test_tem = np.concatenate(pre_test_tem, axis = 0)
            #print (pre_test_tem.shape)
            Y_tes_true = np.concatenate(Y_tes_true, axis = 0)
            
            c_tes = np.sqrt(c_tes / num_test)
            #c_tes_h = c_tes_h / num_test
            
            print("Test RMSE: ", c_tes)
            #print("predic step: ", cost_by_hor)

            if c_val < best_val:
                best_val = c_val
                #saver.save(sess, './bikesharing_graph_2_th_point1')
                test_error = c_tes
                
                traing_error = avg_cost#np.sqrt(avg_cost)
                early_stop_k = 0 # reset to 0
                #print (pred_tes1)
                predic_res = pre_test_tem
                Y_true = Y_tes_true
                #predic_step = cost_by_hor

            # early stopping
            if c_val >= best_val:
                early_stop_k += 1

            # threshold
            if early_stop_k == early_stop_th:
              #  print ("early stopping...")
                break
            

        print("epoch is ", epoch)
        print("training RMSE is ", traing_error)
        print("Optimization Finished! the lowest validation RMSE is ", best_val)#(np.sqrt(best_val)))
        print("The test RMSE is ", test_error)#(np.sqrt(test_error)))
    
    #test_Y = Y_test
    #test_error = np.sqrt(test_error)
    return test_error, predic_res, Y_true#, A1#, predic_step#, A2

In [12]:
# change hidden number
# change batch_size
# set size
#from bayes_opt import BayesianOptimization
import datetime
#freq_max = 12
#time_step = 12
learning_rate = 0.002
decay = 0.9 
batch_size = 500
num_hidden = 100 # number of hiddent units in LSTM Cell
early_stop_th = 150
training_epochs = 1000
keep = 1#0.2
#time_step_max = 10

sn = 272 # station num
#test = 2000 
reg1 = 0.05#0.05
reg2 = 0.05#0.1
reg3 = 0.05
frequency = 24
horizon = 1
lstm_steps = 3  # number of lstm cells
n_hidden_vec1 = 10
n_hidden_vec2 = 5
n_hidden_vec3 = 10
n_hidden_vec4 = 10
#num = 0
#All_pred = np.empty([2000, 207])
#All_Y = np.empty([2000, 207])

#24*90
#step = 0
#gap = 100
#training = 0.7
#validation = 0.1
#test = 0.2

#gcn_corr_eval(7, 0.01, 0.5, 100, 0.4, 10, 5, 5, 0.2, 50, 500)



rep = 1 # repeating times

#total_sn = 0
#num_iter = 50
#init_points = 200


# stdbscan
#spatial_threshold = 300
#temporal_threshold = 300
#min_neighbors = 1 # number of neighbor

#frequency2 = skip1 + freq_max + training

#while step < 2000:

#hourly_bike_cluster = hourly_bike
best = -10000
pre_best = []
test_Y_best = []
test_error_best = 1000
A1_best = []
# A2_best = []
for i in range(rep):
    a = datetime.datetime.now()
    test_error, predic_res, Y_true = gcn_corr_final(frequency, horizon, learning_rate, decay, batch_size, n_hidden_vec1,
                                                                n_hidden_vec2, n_hidden_vec3, keep, early_stop_th, training_epochs, reg1, reg2)
    #val_error, predic_res, test_Y,test_error=gcn_corr_final(a['frequency'], a['learning_rate'], a['decay'], a['batch_size'], a['n_hidden_vec1'], a['n_hidden_vec2'], a['n_hidden_vec3'], a['keep'], a['early_stop_th'], a['training_epochs'], a['reg'])
    #print ("finished A running: ", i)
    b = datetime.datetime.now()
    print(b-a)
    

    #total_sn = total_sn + sn

    #total_error = np.sqrt(np.mean((All_pred[0:(step+gap),0:total_sn] - All_Y[0:(step+gap),0:total_sn])**2))

    #print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the cluster now is: ", c)
    #print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the val error of this cluster now is: ", best)
    #print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the test error by this cluster now is: ", total_error)
    #step = step + gap
    #skip1 = skip1 + gap
    
    #np.savetxt("prediction_2.csv", All_pred, delimiter = ',')
    #np.savetxt("prediction_Y_2.csv", All_Y, delimiter = ',')
    

Epoch: 0001 Training RMSE= 8.299416202
Validation RMSE:  7.580002447948211
Test RMSE:  6.705666653733698
Epoch: 0002 Training RMSE= 7.436435918
Validation RMSE:  7.386007596484858
Test RMSE:  6.475329752298409
Epoch: 0003 Training RMSE= 7.236388247
Validation RMSE:  7.32281848593852
Test RMSE:  6.327300142758965
Epoch: 0004 Training RMSE= 7.065989458
Validation RMSE:  6.975596532983168
Test RMSE:  6.020240816134595
Epoch: 0005 Training RMSE= 6.850644120
Validation RMSE:  6.7757945464039055
Test RMSE:  5.8079438759900555
Epoch: 0006 Training RMSE= 6.658961171
Validation RMSE:  6.593882844138136
Test RMSE:  5.605094870616014
Epoch: 0007 Training RMSE= 6.443181370
Validation RMSE:  6.416954141427694
Test RMSE:  5.422436072261587
Epoch: 0008 Training RMSE= 6.304944727
Validation RMSE:  6.262239069271115
Test RMSE:  5.256020000927754
Epoch: 0009 Training RMSE= 6.153715404
Validation RMSE:  6.107630409850258
Test RMSE:  5.0965355587585845
Epoch: 0010 Training RMSE= 5.971322389
Validation RMS

Epoch: 0079 Training RMSE= 3.267081535
Validation RMSE:  3.280807483443037
Test RMSE:  2.555761548968199
Epoch: 0080 Training RMSE= 3.252055479
Validation RMSE:  3.276403189857848
Test RMSE:  2.5579767546115777
Epoch: 0081 Training RMSE= 3.244679663
Validation RMSE:  3.297369029558239
Test RMSE:  2.557757082124998
Epoch: 0082 Training RMSE= 3.240966133
Validation RMSE:  3.2730828505069893
Test RMSE:  2.5606135193090442
Epoch: 0083 Training RMSE= 3.233300927
Validation RMSE:  3.2765199082734777
Test RMSE:  2.5488293176407053
Epoch: 0084 Training RMSE= 3.224243435
Validation RMSE:  3.312825420877293
Test RMSE:  2.59509885713591
Epoch: 0085 Training RMSE= 3.226860531
Validation RMSE:  3.2542164667269593
Test RMSE:  2.531860089659545
Epoch: 0086 Training RMSE= 3.214934627
Validation RMSE:  3.260148653871237
Test RMSE:  2.5455775086084236
Epoch: 0087 Training RMSE= 3.209155481
Validation RMSE:  3.24586772476082
Test RMSE:  2.5264548104761446
Epoch: 0088 Training RMSE= 3.206157068
Validation

Epoch: 0157 Training RMSE= 2.848209634
Validation RMSE:  2.975077541526865
Test RMSE:  2.3569365525686448
Epoch: 0158 Training RMSE= 2.832907001
Validation RMSE:  2.919795494789302
Test RMSE:  2.267463766626564
Epoch: 0159 Training RMSE= 2.829492592
Validation RMSE:  2.9322911810775727
Test RMSE:  2.2957288386547203
Epoch: 0160 Training RMSE= 2.825411014
Validation RMSE:  2.9070914291226098
Test RMSE:  2.2677124667067936
Epoch: 0161 Training RMSE= 2.821934497
Validation RMSE:  2.903186290738662
Test RMSE:  2.2550390692453846
Epoch: 0162 Training RMSE= 2.806874437
Validation RMSE:  2.8809553123922345
Test RMSE:  2.239606761070555
Epoch: 0163 Training RMSE= 2.805826441
Validation RMSE:  2.9186479424026603
Test RMSE:  2.2851789554869915
Epoch: 0164 Training RMSE= 2.806229030
Validation RMSE:  2.887910362082297
Test RMSE:  2.2484088808402793
Epoch: 0165 Training RMSE= 2.792082407
Validation RMSE:  2.8918868120330252
Test RMSE:  2.2671395994104278
Epoch: 0166 Training RMSE= 2.789680675
Vali

Epoch: 0235 Training RMSE= 2.542600382
Validation RMSE:  2.66673739657379
Test RMSE:  2.1058978066451677
Epoch: 0236 Training RMSE= 2.535182446
Validation RMSE:  2.6725332776559063
Test RMSE:  2.1165803834144725
Epoch: 0237 Training RMSE= 2.530195895
Validation RMSE:  2.6646858224453034
Test RMSE:  2.102955253858771
Epoch: 0238 Training RMSE= 2.534483516
Validation RMSE:  2.659003883403454
Test RMSE:  2.0940938496431167
Epoch: 0239 Training RMSE= 2.531700436
Validation RMSE:  2.6635709292271734
Test RMSE:  2.1101707935361693
Epoch: 0240 Training RMSE= 2.524339868
Validation RMSE:  2.6588661101418913
Test RMSE:  2.104709835259657
Epoch: 0241 Training RMSE= 2.521839073
Validation RMSE:  2.6482862356194015
Test RMSE:  2.092481727224071
Epoch: 0242 Training RMSE= 2.521201607
Validation RMSE:  2.6594543875512007
Test RMSE:  2.104037589341154
Epoch: 0243 Training RMSE= 2.517071383
Validation RMSE:  2.678529294454613
Test RMSE:  2.1329998730526656
Epoch: 0244 Training RMSE= 2.514132660
Valida

Epoch: 0313 Training RMSE= 2.392896725
Validation RMSE:  2.565612321792111
Test RMSE:  2.0353607543585475
Epoch: 0314 Training RMSE= 2.385560557
Validation RMSE:  2.5556538005501843
Test RMSE:  2.0314608977946502
Epoch: 0315 Training RMSE= 2.393471585
Validation RMSE:  2.5606563029704796
Test RMSE:  2.039430007847393
Epoch: 0316 Training RMSE= 2.385072123
Validation RMSE:  2.565153097599349
Test RMSE:  2.0461479235188875
Epoch: 0317 Training RMSE= 2.389117769
Validation RMSE:  2.5761127826660664
Test RMSE:  2.0503712707524686
Epoch: 0318 Training RMSE= 2.384132932
Validation RMSE:  2.5599724469341245
Test RMSE:  2.0447087509307935
Epoch: 0319 Training RMSE= 2.382995373
Validation RMSE:  2.5699104064138147
Test RMSE:  2.056437874533373
Epoch: 0320 Training RMSE= 2.384542676
Validation RMSE:  2.5623113516241043
Test RMSE:  2.0389883385980068
Epoch: 0321 Training RMSE= 2.379616793
Validation RMSE:  2.568239645832452
Test RMSE:  2.0564171215889107
Epoch: 0322 Training RMSE= 2.378339633
Val

...

Epoch: 0625 Training RMSE= 2.148998795
Validation RMSE:  2.4398470118510307
Test RMSE:  1.9623710893703428
Epoch: 0626 Training RMSE= 2.148545130
Validation RMSE:  2.470373754447658
Test RMSE:  1.9876659617918817
Epoch: 0627 Training RMSE= 2.148208426
Validation RMSE:  2.4614831470902088
Test RMSE:  1.9859273800715875
Epoch: 0628 Training RMSE= 2.145620118
Validation RMSE:  2.4600676679608533
Test RMSE:  1.970628614122904
Epoch: 0629 Training RMSE= 2.147698723
Validation RMSE:  2.4420257514795543
Test RMSE:  1.9658030752230724
Epoch: 0630 Training RMSE= 2.146322106
Validation RMSE:  2.4449061339444995
Test RMSE:  1.9627751558416908
Epoch: 0631 Training RMSE= 2.144514073
Validation RMSE:  2.4583882040161003
Test RMSE:  1.9829558757291617
Epoch: 0632 Training RMSE= 2.145131059
Validation RMSE:  2.4638279727653063
Test RMSE:  1.982457383312306
Epoch: 0633 Training RMSE= 2.144612231
Validation RMSE:  2.4638222876656006
Test RMSE:  1.9797942990080781
Epoch: 0634 Training RMSE= 2.143920684
V

Epoch: 0702 Training RMSE= 2.119818032
Validation RMSE:  2.4360120829187024
Test RMSE:  1.9552503237502719
Epoch: 0703 Training RMSE= 2.117812407
Validation RMSE:  2.4694850648195508
Test RMSE:  1.9894976089810223
Epoch: 0704 Training RMSE= 2.117901418
Validation RMSE:  2.437635833672342
Test RMSE:  1.9581927052861183
Epoch: 0705 Training RMSE= 2.118815494
Validation RMSE:  2.4527139258530886
Test RMSE:  1.9774860742322178
Epoch: 0706 Training RMSE= 2.116596258
Validation RMSE:  2.458579711098192
Test RMSE:  1.9726943966038228
Epoch: 0707 Training RMSE= 2.115545682
Validation RMSE:  2.4606636500502437
Test RMSE:  1.981393240916101
Epoch: 0708 Training RMSE= 2.117217528
Validation RMSE:  2.4612095515936416
Test RMSE:  1.9773868304397353
Epoch: 0709 Training RMSE= 2.118248553
Validation RMSE:  2.448599534798637
Test RMSE:  1.9627496621298386
Epoch: 0710 Training RMSE= 2.115710277
Validation RMSE:  2.449657930121954
Test RMSE:  1.9656067839780624
Epoch: 0711 Training RMSE= 2.115340144
Val

Epoch: 0780 Training RMSE= 2.095901622
Validation RMSE:  2.4384906418546284
Test RMSE:  1.9546141631276148
Epoch: 0781 Training RMSE= 2.092973115
Validation RMSE:  2.4305968959259627
Test RMSE:  1.952083538860146
Epoch: 0782 Training RMSE= 2.094949487
Validation RMSE:  2.4402657492330384
Test RMSE:  1.958204865469122
Epoch: 0783 Training RMSE= 2.093390366
Validation RMSE:  2.4451322641463205
Test RMSE:  1.9666368709209086
Epoch: 0784 Training RMSE= 2.093907749
Validation RMSE:  2.429447297123825
Test RMSE:  1.9487964577853687
Epoch: 0785 Training RMSE= 2.092802277
Validation RMSE:  2.44575284460441
Test RMSE:  1.9656738438928323
Epoch: 0786 Training RMSE= 2.092594617
Validation RMSE:  2.451916244980628
Test RMSE:  1.9648624214513801
Epoch: 0787 Training RMSE= 2.093305529
Validation RMSE:  2.439523957005275
Test RMSE:  1.956164931620112
Epoch: 0788 Training RMSE= 2.091435198
Validation RMSE:  2.435971808054296
Test RMSE:  1.960647652774558
Epoch: 0789 Training RMSE= 2.091857661
Validati

Epoch: 0858 Training RMSE= 2.074743198
Validation RMSE:  2.456264018244453
Test RMSE:  1.988025071703662
Epoch: 0859 Training RMSE= 2.071848072
Validation RMSE:  2.437985860868078
Test RMSE:  1.9656359855696626
Epoch: 0860 Training RMSE= 2.074609284
Validation RMSE:  2.454192275629889
Test RMSE:  1.9711033062243963
Epoch: 0861 Training RMSE= 2.073220680
Validation RMSE:  2.4455990608179214
Test RMSE:  1.9684394712630529
Epoch: 0862 Training RMSE= 2.073462506
Validation RMSE:  2.431562957136099
Test RMSE:  1.9577313235077318
Epoch: 0863 Training RMSE= 2.072680010
Validation RMSE:  2.4388743463018456
Test RMSE:  1.9634601431061147
Epoch: 0864 Training RMSE= 2.072248163
Validation RMSE:  2.4392858589924726
Test RMSE:  1.9568997641610497
Epoch: 0865 Training RMSE= 2.071963329
Validation RMSE:  2.449623646342515
Test RMSE:  1.9713745485677403
Epoch: 0866 Training RMSE= 2.072207949
Validation RMSE:  2.432437897778434
Test RMSE:  1.9576278355877943
Epoch: 0867 Training RMSE= 2.072625854
Valid

Epoch: 0935 Training RMSE= 2.056743855
Validation RMSE:  2.4260491117300664
Test RMSE:  1.9547354353066835
Epoch: 0936 Training RMSE= 2.056409529
Validation RMSE:  2.4341519176432898
Test RMSE:  1.9640039731138952
Epoch: 0937 Training RMSE= 2.056763706
Validation RMSE:  2.4397949028194543
Test RMSE:  1.9613334742300104
Epoch: 0938 Training RMSE= 2.054757585
Validation RMSE:  2.43332919279382
Test RMSE:  1.963164520133064
Epoch: 0939 Training RMSE= 2.057870151
Validation RMSE:  2.4273796773804857
Test RMSE:  1.9553417138637472
Epoch: 0940 Training RMSE= 2.055584611
Validation RMSE:  2.44544136755926
Test RMSE:  1.9806694365476365
Epoch: 0941 Training RMSE= 2.055409208
Validation RMSE:  2.4327008980674463
Test RMSE:  1.9581267893880734
Epoch: 0942 Training RMSE= 2.055539013
Validation RMSE:  2.419777620340196
Test RMSE:  1.9508525751325434
Epoch: 0943 Training RMSE= 2.055580695
Validation RMSE:  2.4389595405895172
Test RMSE:  1.96718703399483
Epoch: 0944 Training RMSE= 2.054407775
Valida

In [13]:
np.savetxt("../../data/nyc_bike/lstm_gcnn_prediction_1.95.csv", predic_res, delimiter = ',')
np.savetxt("../../data/nyc_bike/lstm_gcnn_prediction_1.95_Y.csv", Y_true, delimiter = ',')

In [ ]:

# learning rate to 0.005, not decreasing, bouncing
# change it to 0.002

In [24]:
X_whole = []
Y_whole = []

x_offsets = np.sort(
    # np.concatenate(([-week_size + 1, -day_size + 1], np.arange(-11, 1, 1)))
    np.concatenate((np.arange(-frequency+1, 1, 1),))
)
# Predict the next one hour
y_offsets = np.sort(np.arange(1, 1+ horizon, 1))

min_t = abs(min(x_offsets))
max_t = abs(raw_data.shape[0] - abs(max(y_offsets)))  # Exclusive
for t in range(min_t, max_t):
    x_t = raw_data.iloc[t + x_offsets, 0:sn].values.flatten('F')
    y_t = raw_data.iloc[t + y_offsets, 0:sn].values.flatten('F')
    X_whole.append(x_t)
    Y_whole.append(y_t)

X_whole = np.stack(X_whole, axis=0)
time_step = int(X_whole.shape[1] / sn)
#X_whole = np.reshape(X_whole, [X_whole.shape[0], sn, time_step])
Y_whole = np.stack(Y_whole, axis=0)

i = lstm_steps
X_whole_lstm = []
Y_whole_lstm = []

while i < X_whole.shape[0]:
    X_whole_lstm.append(X_whole[i-lstm_steps:i,:])
    Y_whole_lstm.append(Y_whole[i])
    i = i + 1

X_whole_lstm = np.stack(X_whole_lstm, axis = 0) # (34239, 10, 207, 12)
Y_whole_lstm = np.stack(Y_whole_lstm, axis = 0) # (34239, 2484)
#print (Y_whole_lstm.shape)
'''
time_step = int(time_step) #

#i = time_step
#X_whole = np.zeros(shape = (raw_data.shape[0] - time_step, sn*time_step), dtype = np.float)
#Y_whole = np.zeros(shape = (raw_data.shape[0] - time_step, sn), dtype = np.float)

while i < raw_data.shape[0]:
    X_whole[i - time_step, ] = raw_data.iloc[(i - time_step):i, 0:sn].values.flatten('F') # 'F' flatten by column, default:flatten by row 0, 1, 2...7
    Y_whole[i - time_step, ] = raw_data.iloc[i, 0:sn]
    i = i + 1
'''


n_input = sn # station number
n_input_vec = n_input * frequency # 207 * frequency
n_A_vec = n_input * n_input
n_output_vec = Y_whole_lstm.shape[1] # each row represent a result
#print (n_output_vec)


num_samples = X_whole_lstm.shape[0]
num_train = 20000 # Note here actually we use the first 20000 to train the model. The paper mentioned "22304" need to be corrected.
num_val = 2000
num_test = 2000
#skip = skip1 + freq_max - time_step#time_step_max - time_step # to make sure the testing datasets are the same although the frequency could be different

X_training = X_whole_lstm[:num_train, :]
Y_training = Y_whole_lstm[:num_train, :]

# shuffle
perm = np.arange(X_whole_lstm.shape[0])
np.random.shuffle(perm)
X_training = X_whole_lstm[perm]
Y_training = Y_whole_lstm[perm]

#print (type(X_training))
#X_training = random.Random(6).shuffle(X_training)
#Y_training = random.Random(6).shuffle(Y_training)

X_val = X_whole_lstm[num_train:num_train+num_val, :]
Y_val = Y_whole_lstm[num_train:num_train+num_val, :]
#A_val = A_whole[0+training:0+training+validation, :]

X_test = X_whole_lstm[num_train+num_val:num_train+num_val+num_test, :]
Y_test = Y_whole_lstm[num_train+num_val:num_train+num_val+num_test, :]

In [30]:
X_test.shape

(2000, 3, 6528)

In [34]:
Y_test

array([[3, 0, 2, ..., 0, 2, 2],
       [4, 0, 2, ..., 0, 0, 1],
       [2, 2, 0, ..., 0, 1, 1],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [35]:
Y_test.shape

(2000, 272)

In [37]:
np.reshape(X_test[0, 2, :], (272, 24))

array([[ 4,  2,  3, ...,  5,  4, 10],
       [ 6,  3,  7, ..., 13,  9,  2],
       [ 3,  0,  0, ...,  3,  6, 23],
       ...,
       [ 1,  0,  2, ...,  0,  0,  0],
       [ 0,  0,  1, ...,  0,  1,  5],
       [ 0,  1,  1, ...,  2,  0,  2]])

In [41]:
Y_test[0, :]

array([ 3,  0,  2,  5,  3,  4,  3,  9,  8,  4,  5,  8,  3,  5,  7,  8,  7,
        5,  3,  1,  5,  4,  6,  3,  0,  2,  0,  2,  1,  8, 14,  0,  1,  2,
        0,  3,  0, 14, 14,  9,  8,  9,  1,  1,  1,  5,  9,  3,  4,  2,  4,
        5,  1,  6,  4,  0,  3,  8,  3,  2,  9,  2,  9,  1,  1,  2,  1,  0,
        3,  1,  6,  0,  0,  0,  5,  3,  0,  5,  5,  2,  3,  0,  4,  0,  2,
        2, 11,  5, 13,  1,  3,  0,  3,  4,  5,  2,  5,  5,  3,  5,  3,  3,
        5,  3,  2,  7,  2,  3,  2,  2,  2,  2, 16,  7,  1,  2,  1,  1,  5,
        2,  5,  0,  3,  4,  2,  0,  1,  3,  2,  3,  5,  4,  0,  3,  7,  8,
        4,  2,  3,  4,  1,  7,  7,  3,  4, 10,  5,  2,  0,  4,  1,  1,  2,
        0,  0,  0,  5,  1, 15, 17,  3,  4,  0,  5,  2,  7,  0,  6,  5,  4,
        0,  1,  0,  6,  1,  1,  4,  4,  2,  0,  3,  2,  6,  1, 19,  7,  3,
        1,  0,  1,  0,  2,  1,  0,  3,  0,  7,  1,  2,  1,  1,  1,  2,  4,
        3,  0,  1,  0,  5,  3,  9,  7,  0,  5,  2,  4,  7,  7, 11, 19,  7,
        2,  9,  3,  1,  7

In [23]:
X_training[0, 1, :]

array([[0, 0, 3, ..., 1, 0, 1],
       [0, 0, 3, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])